# Crear Target

## Ejercicio 1:
Cree el archivo `competencia_01.csv`, usando el `competencia_01_crudo.csv` con una variable adicional llamada `clase_ternaria`, que contenga las categorías **CONTINUA**, **BAJA+1** y **BAJA+2** 

### Ayudita 

Para practicar el muy útil y necesario lenguaje **SQL**, vamos a utilizar una base de datos **OLAP** llamada **DuckDB**. 

La documentación la puede encontrar [aquí](https://duckdb.org/docs/archive/0.8.1/sql/introduction)
Procedemos a instalarla, esto se debe ejecutar una sola vez

Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb** 

In [1]:
!pip install jupysql
!pip install duckdb-engine


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install duckdb

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install pandas


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

Y ya podemos usar **SQL** dentro de una notebook!

In [5]:
%%sql
select 'hola mundo' 

,'hola mundo'
0,hola mundo


Para cargar el archivo `.csv` a una tabla:

In [5]:
%%sql
create or replace table competencia_01 as 
select 
    *
from read_csv_auto('C:\Users\Kalou\OneDrive\Escritorio\DMEyF\datasets\competencia_01_crudo.csv')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [7]:
%sql select * from competencia_01 limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,202103,1,0,0,50,197,14468.81,125765.29,2389.82,...,0.0,7,0.00,-114954.00,0.00,5938,101050.66,68,0,71811.06
1,29184630,202103,1,0,0,59,322,11901.57,74158.93,18750.68,...,0.0,1,0.00,-40330.15,17.59,4089,26834.09,7,0,3894.36
2,29185433,202103,1,0,0,68,268,847.15,21672.47,481.62,...,0.0,21,4692.00,-1173.00,0.00,7829,1651.36,3,0,1560.09
3,29185587,202103,1,0,0,79,322,4976.94,47735.98,1839.31,...,0.0,1,0.00,-15988.67,0.00,7580,30025.29,11,0,1700.85
4,29185646,202103,1,0,0,60,257,2860.45,37800.71,4035.40,...,0.0,21,380616.14,-97383.25,0.00,7827,359610.70,31,0,15600.90


In [8]:
%%sql
select 
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes 
                           -- y lo guarda en un campo llamado cantidad
from competencia_01
group by foto_mes

,foto_mes,cantidad
0,202103,163324
1,202104,163637
2,202105,164102


Perfecto, ahora cree una nueva tabla con la variable adicional que se le pide. Algunas funciones que le pueden ser útiles:  [where](https://duckdb.org/docs/sql/query_syntax/where), [left join](https://duckdb.org/docs/sql/query_syntax/from), [case statement](https://duckdb.org/docs/sql/expressions/case)



In [19]:
%%sql --save tabla_202103, tabla_202104, tabla_202105 --no-execute
with tabla_202103 as (select numero_de_cliente from competencia_01 
     where foto_mes = '202103'), 
tabla_202104 as (select numero_de_cliente from competencia_01 
     where foto_mes = '202104'
),
tabla_202105 as (select numero_de_cliente from competencia_01 
     where foto_mes = '202105'
)



Skipping execution...

In [6]:
%%sql 
CREATE OR REPLACE TABLE competencia_01_v2 AS
SELECT n.*,r.*
FROM competencia_01 n
LEFT JOIN (select numero_de_cliente,case 
  when numero_de_cliente in (select numero_de_cliente from competencia_01 
     where foto_mes = '202103') and  numero_de_cliente in (select numero_de_cliente from competencia_01 
     where foto_mes = '202104') and numero_de_cliente in (select numero_de_cliente from competencia_01 
     where foto_mes = '202105') then 'CONTINUA'
  when numero_de_cliente in (select numero_de_cliente from competencia_01 
     where foto_mes = '202103') and numero_de_cliente in (select numero_de_cliente from competencia_01 
     where foto_mes = '202104') and numero_de_cliente not in (select numero_de_cliente from competencia_01 
     where foto_mes = '202105') then 'BAJA+2'
  when numero_de_cliente in (select numero_de_cliente from competencia_01 
     where foto_mes = '202103') and numero_de_cliente not in (select numero_de_cliente from competencia_01 
     where foto_mes = '202104') then 'BAJA+1'
   when numero_de_cliente not in (select numero_de_cliente from competencia_01 
     where foto_mes = '202103') and numero_de_cliente  in (select numero_de_cliente from competencia_01 
     where foto_mes = '202104') and numero_de_cliente in (select numero_de_cliente from competencia_01 
     where foto_mes = '202105') then 'CONTINUA'
   when numero_de_cliente not in (select numero_de_cliente from competencia_01 
     where foto_mes = '202103') and numero_de_cliente in (select numero_de_cliente from competencia_01 
     where foto_mes = '202104') and numero_de_cliente not in (select numero_de_cliente from competencia_01 
     where foto_mes = '202105') then 'BAJA+1'
   when numero_de_cliente not in (select numero_de_cliente from competencia_01 
     where foto_mes = '202103') and numero_de_cliente not in (select numero_de_cliente from competencia_01 
     where foto_mes = '202104') and numero_de_cliente in (select numero_de_cliente from competencia_01 
     where foto_mes = '202105') then 'CONTINUA'
  else 'revisar' end as Etiqueta
    from competencia_01 
    where foto_mes = '202103') as r
on n.numero_de_cliente = r.numero_de_cliente

,Success


In [7]:
%%sql 

COPY competencia_01_v2 TO 'C:\Users\Kalou\OneDrive\Escritorio\DMEyF\datasets\competencia_01.csv' (HEADER, DELIMITER ',');


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


## Ejercicio 1.1

* ¿Cuál es la nominalidad de cada clase?
* ¿Cuál es la proporción del target?

In [ ]:
%%sql
-- introduzca sus queries

Para guardar a un **.csv** simplemente debe ejecutar la siguiente sentencia

In [ ]:
%%sql
COPY competencia_01 TO '/home/aleb/dmeyf23/datasets/competencia_01.csv' (FORMAT CSV, HEADER)

## Ejercicio 2 - Avanzado
Use ahora el archivo `ejercicio_target.csv` y calcule para todos los clientes en todos los periodos su **clase_ternaria** al mismo tiempo

In [12]:
%%sql
create or replace table competencia_target as 
select 
    *
from read_csv_auto('C:\Users\Kalou\OneDrive\Escritorio\DMEyF\datasets\ejercicio_target.csv')

,Success


In [16]:
%%sql --save first_cte 

SELECT periodo
    , nombre
    , CASE WHEN (lead(nombre, 2,0) = nombre) AND (lead(periodo, 2,0) - periodo = 2) THEN 'Continua'
            WHEN (lead(nombre, 1,0) = nombre) AND (lead(periodo, 1,0) - periodo > 1) THEN 'Baja+1'
            WHEN (lead(nombre, 1,0) <> nombre) THEN 'Baja+1'
            WHEN (lead(nombre, 2,0) = nombre) AND (lead(periodo, 2,0) - periodo > 2) THEN 'Baja en 2 meses'
            WHEN (lead(nombre, 2,0) <> nombre) THEN 'Baja en 2 meses'
            END AS 'clase_ternaria'
from competencia_target
order by nombre, periodo asc

RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(duckdb.CatalogException) Catalog Error: Scalar Function with name lead does not exist!
Did you mean "lpad"?
LINE 3:     , CASE WHEN (lead(nombre...
                         ^
[SQL: SELECT periodo
    , nombre
    , CASE WHEN (lead(nombre, 2,0) = nombre) AND (lead(periodo, 2,0) - periodo = 2) THEN 'Continua'
            WHEN (lead(nombre, 1,0) = nombre) AND (lead(periodo, 1,0) - periodo > 1) THEN 'Baja+1'
            WHEN (lead(nombre, 1,0) <> nombre) THEN 'Baja+1'
            WHEN (lead(nombre, 2,0) = nombre) AND (lead(periodo, 2,0) - periodo > 2) THEN 'Baja en 2 meses'
            WHEN (lead(nombre, 2,0) <> nombre) THEN 'Baja en 2 meses'
            END AS 'clase_ternaria'
from competencia_target
order by nombre, periodo asc]
(Background on this error at: https://sqlalche.